In [1]:
from os import listdir
from os.path import isfile
from fnmatch import fnmatch
from math import isnan, ceil
from pprint import pprint
from statistics import median
from re import match, search

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties

In [2]:
def get_ratio(filename, mw=False):
    i = 0
    if mw:
        i = 1
    return filename.split('_')[i].replace('ratio','')

In [3]:
def get_run(filename, mw=False):
    i = 1
    if mw:
        i = 2
    return int(filename.split('_')[i].replace('run',''))

In [4]:
def get_vcs(filename, mw=False):
    i = 2
    if mw:
        i = 3
    return int(filename.split('_')[i].replace('vclients',''))

In [5]:
def get_worker_threads(filename, mw=False):
    i = 3
    if mw:
        i = 4 
        return int(filename.split('_')[i].replace('workerthreads', '').replace('.log', ''))
    return int(filename.split('_')[i].replace('workerthreads', ''))

In [6]:
def get_client(filename):
    return int(filename.split('_')[4].replace('client', ''))

In [7]:
def get_instance(filename):
    return int(filename.split('_')[-1].replace('instance','').replace('.log', ''))

In [8]:
def get_mw(filename):
    return int(filename.split('_')[0].replace('stats', ''))

In [9]:
def roundup(x):
    return int(ceil(x / 1.0)) * 1

In [10]:
def dump_csv(df, filename):
    df.to_csv(filename, float_format='%.2f')
    return

In [13]:
timestamp = '20181214144851'
log_dir = 'logs/%s/' % timestamp

logs = [f for f in listdir(log_dir) if fnmatch(f, '*.log')]

mw = '^stats(.*)'
middleware_logs = [f for f in logs if match(mw, f)]

ratios = ['1:0', '0:1']
runs = [x for x in range(1, 3 + 1)] 
instances = [x for x in range(1, 1 + 1)]
threads = 2 
clients = [x for x in range(1, 3 + 1)]
vclients = [32]
worker_threads = [8, 32]
mws = [x for x in range(1, 1 + 1)]

mw_multi_index = [ratios, worker_threads, vclients, runs, mws]
mw_len_index = len(mws) * len(ratios) * len(worker_threads) * len(vclients) * len(runs)
mw_index = pd.MultiIndex.from_product(mw_multi_index, names=['ratios', 'wts', 'vcs', 'runs', 'mws'])
columns = ['Throughput (ops/s)', 'ResponseTime (ms)']
columns += ['Jobs', 'ServiceTime (ms)', 'WaitingTime (ms)', 'QueueSize']
pd.set_option("display.precision", 2)

mw_template = pd.DataFrame([[.0] * len(columns)] * mw_len_index, index=mw_index, columns=columns)

for log_file in middleware_logs:
    ratio = get_ratio(log_file, True) 
    run = get_run(log_file, True)
    vcs = get_vcs(log_file, True)
    wts = get_worker_threads(log_file, True)
    mw = get_mw(log_file)

    with (open(log_dir+log_file, 'r')) as file:
        lines = file.readlines()

        offset = -1
        for i, line in enumerate(lines):
            if "Interval" in line:
                offset = i
             
        table = pd.read_table(log_dir+log_file, skiprows=1, engine='python', nrows=offset -1, skipinitialspace=True, sep=',\s*', index_col=0, names=['Elapsed Time', 'AVG Throughput', 'AVG Queue Size', 'AVG Waiting Time', 'AVG Service Time', 'AVG Response Time', 'AVG SET Response Time', 'AVG GET Response Time', 'AVG MULTI-GET Response Time', '#SETS', '#GETS', '#MULTIGETS', 'GET MissRatio', 'MULTI-GET MissRatio', 'AVG MULTI-GET Size'])
        table = table.astype('float32')
        
        sets = table['#SETS'].sum()
        gets = table['#GETS'].sum()
        multi = table['#MULTIGETS'].sum()
        
        mw_template.loc[(ratio, wts, vcs, run, mw), 'Throughput (ops/s)'] = table['AVG Throughput'].values[1:-1].mean()
        mw_template.loc[(ratio, wts, vcs, run, mw), 'ResponseTime (ms)'] = table['AVG Response Time'].values[1:-1].mean()
        if ratio == '1:0':
            mw_template.loc[(ratio, wts, vcs, run, mw), 'Jobs'] = table['#SETS'].sum()
        else:
            mw_template.loc[(ratio, wts, vcs, run, mw), 'Jobs'] = table['#GETS'].sum()
        mw_template.loc[(ratio, wts, vcs, run, mw), 'ServiceTime (ms)'] = table['AVG Service Time'].values[1:-1].mean()
        mw_template.loc[(ratio, wts, vcs, run, mw), 'WaitingTime (ms)'] = table['AVG Waiting Time'].values[1:-1].mean()
        mw_template.loc[(ratio, wts, vcs, run, mw), 'QueueSize'] = table['AVG Queue Size'].values[1:-1].mean()

        
dump_csv(mw_template, '../out/1_mw_3_server.csv')